In [5]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc

from pathlib import Path

In [6]:
DATOS_BRUTOS = Path("../datos/brutos")

In [7]:
bpl.output_notebook()

Loading BokehJS ...

Para ejecutar estos análisis primero se tiene que ejecutar el notebook del archivo "datos_historicos.ipynb", ese notebook permite descargar los archivos de datos con los que trabaja este notebook.

primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "05/25/2020"

fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

frames = []
for fecha in fechas_csv:
    data = pd.read_csv(DATOS_BRUTOS / fecha, encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    frames.append(data)

In [8]:
frames = []
for archivo in DATOS_BRUTOS.glob("*COVID19MEXICO.csv"):
    data = pd.read_csv(archivo, encoding="latin1", low_memory = False,)
    data["FECHA_ARCHIVO"] = dt.strptime(archivo.name[:6],"%y%m%d").date()
    frames.append(data)

In [9]:
len(frames)

5

In [10]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [11]:
variables_fechas = [col for col in all_data.columns if "FECHA_" in col]

In [12]:
for variable in variables_fechas:
    all_data[variable] = pd.to_datetime(all_data[variable], format = "%Y-%m-%d", errors='coerce')

In [13]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [14]:
confirmados = all_data[all_data["RESULTADO"]==1]
difuntos = confirmados[confirmados["FECHA_DEF"].notna()]

In [15]:
por_fecha = confirmados.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [16]:
fechas_validas = por_fecha.columns

In [17]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [18]:
conf_acum = por_fecha.sum().to_frame("Confirmados").reset_index()

In [19]:
conf_acum["diferencia"] = conf_acum["Confirmados"].diff()

In [20]:
conf_acum[:3]

,FECHA_ARCHIVO,Confirmados,diferencia
0,2020-04-12,4661.0,NaN
1,2020-04-13,5014.0,353.0
2,2020-04-14,5399.0,385.0


In [21]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [22]:
p.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=1,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='1037', ...)

In [23]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i])

In [24]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [25]:
bpl.show(p)

In [26]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [27]:
pday.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=1,top=conf_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='1390', ...)

In [28]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        pday.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],line_alpha=0.3)

In [29]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Casos nuevos reportados'
pday.legend.location = "top_left"

In [30]:
bpl.show(pday)

In [31]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [32]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [33]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [34]:
bpl.show(s)

In [35]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [36]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())*100/(por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [37]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [38]:
bpl.show(s)

In [39]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    conf_sint.name = fecha
    df_fecha.append(conf_sint)

In [40]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-1]]

In [41]:
cambios = valores.pct_change()*100

In [42]:
fechas_casos = cambios.columns

In [43]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [44]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [45]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,10)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [46]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [47]:
bpl.show(s1)

In [48]:
por_fecha_dif = difuntos.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [49]:
fechas_validas = por_fecha_dif.columns

In [50]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [51]:
dif_acum = por_fecha_dif.sum().to_frame("Decesos").reset_index()

In [52]:
dif_acum["diferencia"] = dif_acum["Decesos"].diff()

In [53]:
dif_acum[:3]

,FECHA_ARCHIVO,Decesos,diferencia
0,2020-04-12,296.0,NaN
1,2020-04-13,332.0,36.0
2,2020-04-14,406.0,74.0


In [54]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [55]:
p.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=1,top=dif_acum["Decesos"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos reportados en las conferencias")

GlyphRenderer(id='2842', ...)

In [56]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Defunciones por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i])

In [57]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Decesos confirmados'
p.legend.location = "top_left"

In [58]:
bpl.show(p)

In [59]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [60]:
pday.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=1,top=dif_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='3330', ...)

In [61]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Decesos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],line_alpha=0.3)

In [62]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Decesos nuevos reportados'
pday.legend.location = "top_left"

In [63]:
bpl.show(pday)

In [64]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [65]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    porcentajes = (dif_sint - por_fecha_dif.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [66]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [67]:
bpl.show(s)

In [68]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [69]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    porcentajes = (dif_sint - por_fecha_dif.sum())*100/(por_fecha_dif.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [70]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [71]:
bpl.show(s)

In [72]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    dif_sint.name = fecha
    df_fecha.append(dif_sint)

In [73]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-4]]

In [74]:
cambios = valores.pct_change()*100

In [75]:
fechas_casos = cambios.columns

In [76]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [77]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [78]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,10)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [79]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [80]:
bpl.show(s1)

In [81]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [82]:
pacientes = all_data.groupby(["ID_REGISTRO","ENTIDAD_UM","ENTIDAD_RES","MUNICIPIO_RES","EDAD","EMBARAZO","HABLA_LENGUA_INDIG","DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION","OTRA_COM","CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","MIGRANTE"]).size().reset_index().drop(columns=[0])

In [83]:
varios = pacientes["ID_REGISTRO"].value_counts()[pacientes["ID_REGISTRO"].value_counts()>1].index

In [84]:
varios

Index(['06609a', '144742', '060c21', '05f8ab', '14e6a9', '0fb279', '1d29e5',
       '0dca80', '09f6fa', '063fe9', '0b75b1', '0460d4', '12d480', '167b40',
       '062fa2', '0d41fc', '13a2f7', '073bca', '0de1f3', '041aaf', '03b255',
       '0842d2', '15d77f', '05f621', '1875e1', '011d34', '0f36d2', '118ecb',
       '05f53e', '02e662', '0d7e40', '019dba', '14c960', '0b353b', '092d31',
       '08302a', '1d640b', '1e6cc1', '184620', '1df109', '0fef69', '0c685e',
       '1588d1', '041605', '09cf5d', '147722', '1524fe', '15f363', '0a5d82',
       '1dc610', '0bd0a1', '0e53b3', '0bd7bd', '00a127'],
      dtype='object')

In [85]:
conf_validos = pd.merge(pacientes,por_fecha.reset_index()[~por_fecha.reset_index()["ID_REGISTRO"].isin(varios)],how="right",on="ID_REGISTRO")

In [86]:
conf_filtrados = conf_validos
#conf_filtrados = conf_validos[(conf_validos["ENTIDAD_RES"]==30)]
#conf_filtrados = conf_validos[(conf_validos["ENTIDAD_RES"]==2)&(conf_validos["MUNICIPIO_RES"]==4)]

In [87]:
conf_acum = conf_filtrados[fechas_validas].sum().to_frame("Confirmados").reset_index()

In [88]:
conf_acum["diferencia"] = conf_acum["Confirmados"].diff()

In [89]:
conf_acum[:3]

,index,Confirmados,diferencia
0,2020-04-12,4649.0,NaN
1,2020-04-13,4999.0,350.0
2,2020-04-14,5380.0,381.0


In [90]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [91]:
p.vbar(x=conf_acum["index"],bottom=1,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='5251', ...)

In [92]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [93]:
bpl.show(p)

In [94]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [95]:
s.vbar(x=conf_acum["index"],bottom=1,top=conf_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='5721', ...)

In [96]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Casos nuevos confirmados'
s.legend.location = "top_left"

In [97]:
bpl.show(s)